# Imports

In [ ]:
!pip install torch-scatter -f https://pytorch-geometric.com/whl/torch-2.4.0+cu121.html
!pip install torch-sparse -f https://pytorch-geometric.com/whl/torch-2.4.0+cu121.html
!pip install torch-cluster -f https://pytorch-geometric.com/whl/torch-2.4.0+cu121.html
!pip install torch-spline-conv -f https://pytorch-geometric.com/whl/torch-2.4.0+cu121.html
!pip install torch-geometric
!pip install torch-geometric-temporal
!pip install yfinance
!pip install requests_html
!pip install yahoo_fin

In [ ]:
!pip install --upgrade torch-geometric torch-scatter torch-sparse torch-cluster torch-spline-conv -f https://data.pyg.org/whl/torch-2.4.0+cu121.html

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import yfinance as yf
import pandas as pd
import numpy as np
import json
from torch_geometric.nn import GATConv
from torch_geometric_temporal.nn.recurrent import TGCN

# This will error out, open the TGCN file and replace 'from torch_geometric.utils.to_dense_adj import to_dense_adj' with
#                                                     'from torch_geometric.utils import to_dense_adj'
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
import requests
from tqdm import tqdm
import os

# Load Data

In [ ]:
# Load price data csv
price_data = pd.read_csv("price_data.csv", index_col=0)

# Preprocessing

In [ ]:
from tqdm import tqdm
from sklearn.preprocessing import StandardScaler

In [ ]:
def create_correlation_graph(price_data, corr_min, window_size=20):
    """
    Create rolling correlation-based edge weights
    """
    returns = price_data.pct_change()

    # Calculate rolling correlation
    corr_matrices = []
    for i in range(len(returns) - window_size + 1):
        window = returns.iloc[i:i+window_size]
        corr = window.corr()
        corr_matrices.append(corr)

    # Create edges and weights
    edge_index = []
    edge_weights = []

    for corr_matrix in tqdm(corr_matrices):
        edges = np.argwhere(corr_matrix.to_numpy() > corr_min)

        weights = corr_matrix.to_numpy()[edges[:, 0], edges[:, 1]]
        edge_index.append(torch.tensor(edges).t().contiguous())
        edge_weights.append(torch.tensor(weights))

    return edge_index, edge_weights

def prepare_stock_features(symbols, price_data, pos_bound, neg_bound, scaler=None, window_size=20):
    """Prepare stock features with technical indicators"""
    exceeds_all = []
    misses_all = []
    # Calculate features
    features = []
    for symbol in symbols:
        df = pd.DataFrame()
        # Price momentum
        df['returns'] = price_data[symbol].pct_change()
        df['momentum'] = price_data[symbol].pct_change(window_size)
        # Volatility
        df['volatility'] = df['returns'].rolling(window=window_size).std()
        # Moving averages
        df['ma20'] = price_data[symbol].rolling(window=20).mean()
        df['ma50'] = price_data[symbol].rolling(window=50).mean()

        df['tern'] = 0
        df.loc[df['returns'] > pos_bound * df['volatility'], 'tern'] = 1
        df.loc[-df['returns'] > neg_bound * df['volatility'], 'tern'] = -1
        exceeds_all.append(len(df[df.tern == 1]) / len(df))
        misses_all.append(len(df[df.tern == -1]) / len(df))
        features.append(df)

    print("Exceeds Pricing: ", sum(exceeds_all) / len(exceeds_all))
    print("Misses Pricing: ", sum(misses_all) / len(misses_all))

    # Combine features
    combined_features = np.stack([f.fillna(0).values for f in features], axis=1)
    if scaler is None:
      # Scale features
      scaler = StandardScaler()
      scaled_features = scaler.fit_transform(combined_features.reshape(-1, combined_features.shape[-1]))
      scaled_features = scaled_features.reshape(combined_features.shape)
    else:
      scaled_features = scaler.transform(combined_features.reshape(-1, combined_features.shape[-1]))
      scaled_features = scaled_features.reshape(combined_features.shape)
    scaled_features[:, :, -1] = combined_features[:, :, -1]
    return torch.tensor(scaled_features), scaler

In [ ]:
# Load data
split_date = price_data.index[-126]  # Approximately 126 business days for last 6 months
price_data_train = price_data[price_data.index < split_date]
price_data_test = price_data[price_data.index >= split_date]

In [ ]:
CORR_MIN = 0.4

POSITIVE_BOUND = 0.4
NEGATIVE_BOUND = 0.4

In [ ]:
# Prepare Train Data
print("Preparing training stock features...")
features_train, scaler_train = prepare_stock_features(price_data_train.columns, price_data_train, POSITIVE_BOUND, NEGATIVE_BOUND)
features_train = features_train.to(torch.float32)

print("Creating training correlation matrices...")
edge_index_train, edge_weights_train = create_correlation_graph(price_data_train, CORR_MIN)
edge_index_train = [ei.to(torch.long) for ei in edge_index_train]
edge_weights_train = [ew.to(torch.float32) for ew in edge_weights_train]

# Clip excess train data
features_train = features_train[-len(edge_index_train):]

Preparing stock features...
Exceeds Pricing:  0.31907777040744495
Misses Pricing:  0.313248507507688


In [ ]:
# Prepare Test Data
print("Preparing testing stock features...")
features_test, _ = prepare_stock_features(price_data_test.columns, price_data_test, POSITIVE_BOUND, NEGATIVE_BOUND, scaler=scaler_train)
features_test = features_test.to(torch.float32)

print("Creating testing correlation matrices...")
edge_index_test, edge_weights_test = create_correlation_graph(price_data_test, CORR_MIN)
edge_index_test = [ei.to(torch.long) for ei in edge_index_test]
edge_weights_test = [ew.to(torch.float32) for ew in edge_weights_test]

# Clip excess test data
features_test = features_test[-len(edge_index_test):]

Exceeds Pricing:  0.28134016891781355
Misses Pricing:  0.24850307272667393


100%|██████████| 107/107 [00:04<00:00, 24.42it/s]


In [ ]:
len(features_train)

1114

# Train Model

In [ ]:
class TemporalStockPredictor(nn.Module):
    def __init__(self, node_features, hidden_dim, num_heads=4):
        super(TemporalStockPredictor, self).__init__()

        # Temporal GCN layer
        self.tgcn = TGCN(in_channels=node_features,
                         out_channels=hidden_dim)

        # Graph attention layer
        self.gat = GATConv(
            in_channels=hidden_dim,
            out_channels=hidden_dim,
            heads=num_heads,
            concat=True,
            dropout=0.3
        )

        # Prediction layers
        self.fc1 = nn.Linear(hidden_dim * num_heads, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, 3)

    def forward(self, x, edge_index, edge_weight):
        h = None
        # Process temporal sequence maintaining hidden state
        for t in range(x.size(0)):  # Iterate through time steps
            # Select current time step
            x_t = x[t, :, :]
            edge_index_t = edge_index[t].to(device)
            edge_weight_t = edge_weight[t].to(device)
            # Update TGCN with hidden state
            h = self.tgcn(x_t, edge_index_t, edge_weight_t, H=h)

        # Apply GAT only to final hidden state
        h = self.gat(h, edge_index_t, edge_weight_t)

        # Make predictions
        h = F.relu(self.fc1(h))
        pred = self.fc2(h)

        return pred

In [ ]:
model = TemporalStockPredictor(
    node_features=features_train.shape[-1],
    hidden_dim=64,
    num_heads=8
).to(device)

model

TemporalStockPredictor(
  (tgcn): TGCN(
    (conv_z): GCNConv(6, 64)
    (linear_z): Linear(in_features=128, out_features=64, bias=True)
    (conv_r): GCNConv(6, 64)
    (linear_r): Linear(in_features=128, out_features=64, bias=True)
    (conv_h): GCNConv(6, 64)
    (linear_h): Linear(in_features=128, out_features=64, bias=True)
  )
  (gat): GATConv(64, 64, heads=4)
  (fc1): Linear(in_features=256, out_features=64, bias=True)
  (fc2): Linear(in_features=64, out_features=3, bias=True)
)

Exceeds Pricing:  0.316635446449119
Misses Pricing:  0.28377863222584016


In [ ]:
def train_model(model, train_data, test_data, epochs=100, lr=0.001):
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    # scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
    #     optimizer, mode='min', factor=0.5, patience=5
    # )
    criterion = nn.CrossEntropyLoss()
    train_features, train_edges, train_weights = train_data
    test_features, test_edges, test_weights = test_data

    for epoch in range(epochs):
        model.train()
        total_loss = 0

        # Process in temporal order with sliding window
        for t in tqdm(range(7, len(train_edges)-1)):
            optimizer.zero_grad()

            # Get current timestep data
            x = train_features[t - 7 :t].to(device)
            edge_index = train_edges[t - 7 : t]
            edge_weight = train_weights[t - 7 : t]

            # Forward pass
            pred = model(x, edge_index, edge_weight)

            # Calculate loss using next timestep's ternary labels
            target = train_features[t, :, -1].to(device).long() + 1 # Last feature is the label
            y_one_hot = torch.nn.functional.one_hot(target, 3).to(torch.float64).to(device)
            loss = criterion(pred, y_one_hot)

            # Backward pass
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
            optimizer.step()

            total_loss += loss.item()

        print(f"Epoch {epoch}: Train Loss = {total_loss/(len(train_features) - 7):.4f}")

In [ ]:
train_model(model, (features_train, edge_index_train, edge_weights_train), (features_test, edge_index_test, edge_weights_test), epochs=15, lr=0.001)

 67%|██████▋   | 742/1106 [02:40<01:24,  4.32it/s]

In [ ]:
torch.save(model.state_dict(), "tgcnn-0.4-0.4-2000-1")

# Evaluation

In [ ]:
# Evaluate all test data
with torch.no_grad():

    model.eval()

    features_test = features_test.to(torch.float32)
    edge_index_test = [ei.to(torch.long) for ei in edge_index_test]
    edge_weights_test = [ew.to(torch.float32) for ew in edge_weights_test]

    predictions = []

    # Generate predictions for each time step
    for t in range(7, len(edge_index_test)):
        pred = model(
            features_test[t-7:t].to(device),
            edge_index_test[t-7:t],
            edge_weights_test[t-7:t]
        )
        predictions.append(pred.cpu().numpy())

In [ ]:
# Process predictions
predictions = np.stack(predictions)
tern_predictions = np.argmax(predictions, axis = 2) - 1

In [ ]:
# Load actual percent change and ternary label data
prices_pct_change = price_data.pct_change().fillna(0).to_numpy()[-107:]
tern_prices = features_test[:, :, 5]

In [ ]:
# Count predicted ternary labels v. actual ternary labels
# Count predicted ternary labels v. actual binary labels
# Compute expected return
def generate_counts(tern_predictions, tern_prices, prices_pct_change):
  counts = np.zeros((3, 3))
  bin_counts = np.zeros((3, 2))
  evs = [[], [], []]
  mu_std = np.zeros((3, 3))
  (t, c) = tern_predictions.shape

  for i in range(t):
    for j in range(c):
      # Handle ternary label
      counts[int(tern_predictions[i, j] + 1), int(tern_prices[i, j].item() + 1)] += 1

      # Handle binary labels
      if tern_predictions[i, j] == -1: # We predict it goes down
        evs[0].append(prices_pct_change[i, j])
        if prices_pct_change[i, j] <= 0: # It actually goes down
          bin_counts[0, 0] += 1
        else:
          bin_counts[0, 1] += 1
      elif tern_predictions[i, j] == 1: # We predict it goes up
        evs[2].append(prices_pct_change[i, j])
        if prices_pct_change[i, j] > 0: # It actually goes up
          bin_counts[2, 1] += 1
        else:
          bin_counts[2, 0] += 1
      else:
        evs[1].append(prices_pct_change[i, j])
        if prices_pct_change[i, j] > 0: # It actually goes up
          bin_counts[1, 1] += 1
        else:
          bin_counts[1, 0] += 1

  # Compute expected returns
  for i in range(3):
    mu_std[i, 0] = np.mean(evs[i])
    mu_std[i, 1] = np.std(evs[i]) / np.sqrt(len(evs[i]))
    mu_std[i, 2] = abs((mu_std[i, 0] - prices_pct_change.mean().item()) / mu_std[i, 1])

  return counts, bin_counts, mu_std, prices_pct_change.mean().item()

Mean Pct Change:  tensor(0.0161)
Binary Counts:  [[  5013.   4924.]
 [114164. 114316.]
 [  1018.   2065.]]
Mu Std:  [[4.22898633e-03 4.05405002e-03 2.93331714e+00]
 [1.81876440e-02 1.58791007e-03 1.30161226e+00]
 [1.02718994e-01 5.45905725e-03 1.58632140e+01]]
Counts:  [[ 3230.  3110.  3597.]
 [66965. 84964. 76551.]
 [  552.   931.  1600.]]


In [ ]:
def report(counts, bin_counts, mu_std):
  actual_tern_sums = np.sum(counts, axis = 0)
  pred_sums = np.sum(counts, axis = 1)
  actual_bin_sums = np.sum(bin_counts, axis = 0)

  print("Overall Binary Accuracy", (bin_counts[0, 0] + bin_counts[2, 1]) / (bin_counts[0, 0] + bin_counts[0, 1] + bin_counts[2, 0] + bin_counts[2, 1]))

  print("Binary Negative Recall", (bin_counts[0, 0]) / actual_bin_sums[0])
  print("Binary Positive Recall", (bin_counts[2, 1]) / actual_bin_sums[1])

  print("Binary Negative Precision", (bin_counts[0, 0]) / pred_sums[0])
  print("Binary Positive Precision", (bin_counts[2, 1]) / pred_sums[2])

  print("Overall Ternary Accuracy", (counts[0, 0] + counts[1, 1] + counts[2, 2]) / np.sum(counts))

  print("Ternary Negative Recall", counts[0, 0] / actual_tern_sums[0])
  print("Ternary Approximately Equal Recall", counts[1, 1] / actual_tern_sums[1])
  print("Ternary Positive Recall", counts[2, 2] / actual_tern_sums[2])

  print("Ternary Negative Precision", counts[0, 0] / pred_sums[0])
  print("Ternary Approximately Equal Precision", counts[1, 1] / pred_sums[1])
  print("Ternary Positive Precision", counts[2, 2] / pred_sums[2])

  print(f"Negative Expected Return: {mu_std[0, 0]} ± {mu_std[0, 1]}")
  print(f"Approximately Equal Expected Return: {mu_std[1, 0]} ± {mu_std[1, 1]}")
  print(f"Positive Expected Return: {mu_std[2, 0]} ± {mu_std[2, 1]}")

  print(f"Negative rejects Null Hypothesis with t = ", mu_std[0, 2])
  print(f"Approximately Equal rejects Null Hypothesis with t = ", mu_std[1, 2])
  print(f"Positive rejects Null Hypothesis with t = ", mu_std[2, 2])

counts, bin_counts, mu_std, mean_change = generate_counts(tern_predictions, tern_prices, prices_pct_change)

print("Binary Counts: ", bin_counts)
print("Counts: ", counts)
print("Average Change: ", mean_change)

report(counts, bin_counts, mu_std)

Overall Binary Accuracy 0.5436251920122888
Overall Accuracy 0.3718178053830228
Overall Negative Accuracy 0.5044782127402636
Overall Positive Accuracy 0.6698021407719753
Negative Recall 0.045655646175809576
Approximately Equal Recall 0.9545980562889725
Positive Recall 0.019572344277535843
Negative Precision 0.32504780114722753
Approximately Equal Precision 0.37186624649859945
Positive Precision 0.5189750243269543
